In [5]:
import cv2
import torch
import torch.nn.functional as F

from model_hourglass import StackedHourGlass
from utils import get_args, to_cuda
from visualize import visualize

#from out_to_mesh import get_coords, to_stl

In [6]:
img = cv2.imread('cubic.jpg', cv2.IMREAD_COLOR)
img = cv2.resize(img, (128, 128), interpolation=cv2.INTER_AREA)
img = torch.from_numpy(img.transpose(2, 0, 1)).float()
img = (img - 128) / 128.0
img.shape

torch.Size([3, 128, 128])

In [7]:
saved_model = 'trained_models/hourglass_55_epochs'

model = StackedHourGlass(nChannels=224, nStack=2, nModules=2, numReductions=4, nOutputs=200)
model.cuda()
model.load_state_dict(torch.load(saved_model))
model.eval()

img = to_cuda(img.unsqueeze(0), True)
out = F.sigmoid(model(img))

img = img.squeeze(0).cpu()
out = out.detach().squeeze(0).cpu()

In [8]:
# Visualization

thr, sz = 0.9, 0.25

#x, y, z = get_coords(out.numpy(), sz=sz, thr=thr)
#to_stl("../meshes/out_test", x, y, z)

visualize(img, out, sz=0.25, thr=thr)

Image shape: (3, 128, 128)
Volume shape: (200, 128, 128)
